In [1]:
from ChemstationAPI.ChemstationAPI import ChemstationAPI
from SIA_API.devices.syringe_controller import SyringeController
from SIA_API.devices.valve_selector import ValveSelector
from SIA_API.methods.prepared_methods import PreparedSIAMethods

from tqdm.notebook import tqdm
import pandas as pd
import time

In [2]:
Syringe = SyringeController("COM5", 1000)
Valve = ValveSelector("COM5", 8)
Chemstation = ChemstationAPI()
SIA_methods = PreparedSIAMethods(chemstation_controller=Chemstation, syringe_device=Syringe, valve_device=Valve)

MeOH_port = 5
DI_port = 3

Syringe size: 1000 µL
Syringe resolution: 0.33 µL
Minimum flow rate: 50.0 µL/min
Maximum flow rate: 60000.0 µL/min


ConnectionError: Failed to establish communication with ChemStation.
Please ensure:
1. ChemStation is running
2. Communication macro is loaded: macro C:\Users\hpst\python\SIA-CE\ChemstationAPI\core\ChemPyConnect.mac; Python_Run
3. Communication files path is accessible

In [7]:
Chemstation.system.add_register_reader()

ChemstationError: ChemStation command failed:  ERROR: The command  macro c:\Users\hpst\python\controllers\macros\register_reader.mac  failed to execute. Error message:  Could not open file: C:\USERS\HPST\PYTHON\CONTROLLERS\MACROS\REGISTER_READER.MAC

In [7]:
Valve.position(1)

In [4]:
SIA_methods.system_initialization_and_cleaning()

In [6]:
SIA_methods.prepare_batch_flow(3)

vials = [40, 41]

for vial in vials:
    SIA_methods.batch_fill(vial, 100, 3, flush_needle=20)

In [7]:
Valve.position(1)
CavroX1.valve_in()
CavroX1.aspirate_uL(200)

CavroX1.valve_up()
CavroX1.aspirate_uL(500)
CavroX1.valve_out()
CavroX1.dispense_uL()

CavroX1.valve_in()
CavroX1.aspirate_uL()
CavroX1.valve_out()

Valve.position(1)
CavroX1.dispense_uL()

Valve.position(2)
CavroX1.aspirate_uL(25)
CavroX1.valve_up()
CavroX1.dispense_uL()
CavroX1.valve_out()

In [4]:
CavroX1.valve_in()
Valve.position(1)
CavroX1.aspirate_uL(50)
CavroX1.valve_out()
CavroX1.dispense_uL()

Valve.position(2)
CavroX1.aspirate_uL(25)
CavroX1.valve_up()
CavroX1.dispense_uL()
CavroX1.valve_out()

In [3]:
SIA_methods.prepare_batch_flow(3)

In [ ]:
df = pd.read_excel("samples_info.xlsx")

batch_size = 3

df_batches = [df.iloc[i:i + batch_size] for i in range(0, len(df), batch_size)]

for batch in df_batches:
    for _, sample in batch.iterrows():
        SIA_methods.execute_batch_filling_operation(int(sample.Vial), sample.MeOH, 5)
        print(f"Vial {sample.Vial} is filled by MeOH")
    
    for _, sample in batch.iterrows():
        SIA_methods.execute_batch_filling_operation(int(sample.Vial), int(sample.DI), 3)
        print(f"Vial {sample.Vial} is filled by DI")


In [9]:
Chemstation.sequence.prepare_sequence_table("samples_info.xlsx", sequence_name="SIA_CP_Test",
                                            method_column="Method", sample_name_column="Name",
                                            vial_column="Vial")

In [8]:
Chemstation.ce.load_vial_to_position(50)

Valve.position(2)
CavroX1.aspirate_uL(25)
Valve.position(3)
CavroX1.aspirate_uL(330)
Valve.position(4)
CavroX1.dispense_uL(300)
Valve.position(1)
CavroX1.dispense_uL()

Chemstation.ce.load_vial_to_position(45)
Valve.position(2)
CavroX1.aspirate_uL(500)
Valve.position(4)
CavroX1.dispense_uL()

In [8]:
Chemstation.ce.unload_vial_from_position()

Valve.position(2)
CavroX1.aspirate_uL(15)
Valve.position(4)
CavroX1.dispense_uL()

In [4]:
SIA_methods.homogenize_sample(40, 3500, 30, 20)

In [ ]:
for sample in df.itertuples():

    while Chemstation.system.RC_status() not in ["Idle", "Run"]: #zjištění jestli se může ovládat carousel
        print("wait for carousel")
        time.sleep(2)

    SIA_methods.execute_batch_filling_operation(int(sample.Vial), sample.MeOH, 5) #Naplnění vilky MeOH
    print("add MeOH")
    
    time.sleep(3) #čekání

    SIA_methods.execute_batch_filling_operation(int(sample.Vial), sample.DI, 3) #naplnění vilky DI
    print("add DI")

    if Chemstation.system.method_on():
        while Chemstation.system.get_remaining_analysis_time() > 1.2: #počkání 5 minut před koncem
            time.sleep(5)
        SIA_methods.execute_sample_homogenization(sample.Vial, 2000, 20)
        print("homogenization")
        while Chemstation.system.RC_status() == "Run": #zjištění jestli se může ovládat carousel
            print("wait for end run")
            time.sleep(2)

    else:
        Chemstation.sequence.start() #spuštění sequence
    time.sleep(10)

In [ ]:
    def unload_vial_from_position(self, position: str = "replenishment") -> None:
        """Unload vial from the specified position in CE carousel.
        
        Args:
            position: Position from which to unload the vial.
            
        """
        # Position to command mapping
        position_commands = {
            "inlet": "INLT",
            "outlet": "OUTL",
            "replenishment": "LRPL",
        }
        
        command = position_commands[position]
        self.comm.send(f'WriteModule "{MODULE}", "{command}"')
        time.sleep(3)

In [45]:
import time
from datetime import datetime

def save_time_attempt(sample_name, time_attempt, filename):
    """Uloží čas pokusu do textového souboru"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filename, 'a', encoding='utf-8') as f:
        f.write(f"{timestamp}\t{sample_name}\t{time_attempt:.2f}\n")
    print(f"{timestamp}\t{sample_name}\t{time_attempt:.2f}\n")

# Konstanty
VIAL = 40
METHOD = "NHL3_MD_B"

MEOH = 400
MeOH_port = 5

DI = 100
DI_port = 3

SLEEP_TIME = 600
N_REPETITION = 8

filename = "test.txt"

Chemstation.validation.validate_method_name(METHOD)

# SIA_methods.batch_fill(VIAL, MEOH, MeOH_port)

# for _ in tqdm(range(SLEEP_TIME)):
#     time.sleep(1)

# SIA_methods.batch_fill(VIAL, DI, DI_port)
# time_0 = time.time()

for i in tqdm(range(N_REPETITION)):
    sample_name = f"{i+3}_elution_time_mitra_A_internal_IBU"


    Chemstation.method.execution_method_with_parameters(VIAL, METHOD, sample_name)
    time.sleep(5)
   
    while Chemstation.system.status() != "INJECTING":
        time.sleep(1)
   
    time_attempt = time.time() - time_0
    save_time_attempt(sample_name, time_attempt, filename)  # Uložení času do souboru
    
    while Chemstation.system.get_remaining_analysis_time() > 1.5:
        time.sleep(1)
    
    SIA_methods.homogenize_sample(VIAL, 3500, 30, flush_needle=20)
    
    while Chemstation.system.status() != "PRERUN":
        time.sleep(2)

  0%|          | 0/8 [00:00<?, ?it/s]

2025-07-03 12:56:46	3_elution_time_mitra_A_internal_IBU	1167.62

2025-07-03 13:04:25	4_elution_time_mitra_A_internal_IBU	1627.23

2025-07-03 13:12:05	5_elution_time_mitra_A_internal_IBU	2086.94

2025-07-03 13:19:44	6_elution_time_mitra_A_internal_IBU	2545.59

2025-07-03 13:27:22	7_elution_time_mitra_A_internal_IBU	3003.71

2025-07-03 13:35:01	8_elution_time_mitra_A_internal_IBU	3463.09

2025-07-03 13:42:40	9_elution_time_mitra_A_internal_IBU	3921.46

2025-07-03 13:50:19	10_elution_time_mitra_A_internal_IBU	4380.36



In [44]:
while Chemstation.system.get_remaining_analysis_time() > 1.5:
    time.sleep(1)

SIA_methods.homogenize_sample(VIAL, 3500, 30, flush_needle=20)

In [43]:
CavroX1.volume_counter = 0

In [31]:
import time
from datetime import datetime

def save_time_attempt(sample_name, time_attempt, filename):
    """Uloží čas pokusu do textového souboru"""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    with open(filename, 'a', encoding='utf-8') as f:
        f.write(f"{timestamp}\t{sample_name}\t{time_attempt:.2f}\n")
    print(f"{timestamp}\t{sample_name}\t{time_attempt:.2f}\n")

VIAL = 25
METHOD = "NHL3_MD_B"
sample_name = f"capitainer_30_wait_homogenization_A"

MEOH = 400
MeOH_port = 5

DI = 100
DI_port = 3

SLEEP_TIME = 900
WAIT_TIME = 1600

filename = "test.txt"

Chemstation.validation.validate_method_name(METHOD)

In [36]:
SIA_methods.homogenize_sample(50, 2000, 5, 20)

In [37]:
SIA_methods.batch_fill(VIAL, MEOH, MeOH_port)

for _ in tqdm(range(SLEEP_TIME)):
    time.sleep(1)

SIA_methods.batch_fill(VIAL, DI, DI_port, unload=False)
time_0 = time.time()

SIA_methods.homogenize_sample(VIAL, 3500, 30)

for _ in tqdm(range(WAIT_TIME)):
    time.sleep(1)

SIA_methods.homogenize_sample(VIAL, 3500, 30, flush_needle=20)

# Chemstation.method.execution_method_with_parameters(VIAL, METHOD, sample_name)
# time.sleep(5)

# while Chemstation.system.status() != "INJECTING":
#     time.sleep(1)

# time_attempt = time.time() - time_0
# save_time_attempt(sample_name, time_attempt, filename)

  0%|          | 0/900 [00:00<?, ?it/s]

  0%|          | 0/1600 [00:00<?, ?it/s]

In [15]:
wait = None

if wait:
    print(wait)
    print("haha")

In [23]:
Chemstation.validation.get_vialtable()

{1: True,
 2: True,
 3: False,
 4: False,
 5: True,
 6: True,
 7: True,
 8: True,
 9: True,
 10: True,
 11: False,
 12: False,
 13: False,
 14: False,
 15: False,
 16: False,
 17: False,
 18: False,
 19: False,
 20: True,
 21: True,
 22: True,
 23: True,
 24: False,
 25: False,
 26: False,
 27: False,
 28: False,
 29: False,
 30: False,
 31: False,
 32: False,
 33: False,
 34: False,
 35: False,
 36: False,
 37: False,
 38: False,
 39: False,
 40: False,
 41: False,
 42: False,
 43: False,
 44: False,
 45: True,
 46: True,
 47: False,
 48: True}

In [46]:
SIA_methods.batch_fill(50, 50, MeOH_port, flush_needle=30)

In [29]:
SIA_methods.batch_fill(35, 250, MeOH_port)
# #time.sleep(300)
# SIA_methods.batch_fill(31, 100, DI_port, unload=False)

In [30]:
SIA_methods.homogenize_sample(31, 3000, 30, unload=False)